<a href="https://colab.research.google.com/github/Suweslab/SQL-and-Python/blob/main/sqlite3_staff_answers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#SQL and Python (and a little Pandas)

When it comes to data retrieval and basic statistics, SQL shines, while Python shines for in-depth, flexible exploratory data analysis or data science. What if you could combine the two programming languages into a single application?

We will use SQLite3 which is part of the standard Python 3 package, so nothing to install. Let’s take a quick look at the data types that are available:

| SQL | Python |
|-----|--------|
| NULL — Includes a NULL value | none
| INTEGER — Includes an integer | int
| REAL — Includes a floating-point (decimal) value | float
| TEXT. — Includes text | str
| BLOB. — Includes a binary large object that is stored exactly as input | bytes

In this notebook, we’ll demonstrate

* Loading the library
* Creating and connecting to your database
* Creating database tables
* Adding data
* Querying data
* Deleting data



## Import the library

Let’s start off the tutorial by loading in the library. We can do this by using the following command

In [193]:
import sqlite3

## Create a connection

We use the connect method and pass the name of the database. Let's create an *orders.db*.

In [194]:
#this connects to the database
conn = sqlite3.connect('superheros.db')

We’ve created a new connection object, as well as a new file called orders.db in the directory in which you’re working.


## Create a cursor object

Now that we’ve created a database connection object, our next task is to create a cursor object. A cursor object allows us to execute SQL queries against a database. A cursor acts a middleware between a connection and SQL query

In [195]:
#the cursor is an imaginary pointer that moves in the database
#when the table is created the cursor will be on the 1st row by default
#All queries will be with respect to the current position of the cursor.
#so we create a cursor
cur = conn.cursor() # sometimes useful to think if as 'rows' returned

# Create Table

Use a common 'pattern'

1. Create a SQL command as a string
2. User cursor to execture the command
3. If needed fetch the results

In [196]:
sqldrop='''
DROP TABLE IF EXISTS users;
'''
cur.execute(sqldrop)
cur.fetchall()
conn.commit()



#this is a string that contains the sql command. We are creating a new table with this schema. The table name is users.
# This is the type data that will be accepted(all text)User id will be the primary key
sql = """
CREATE TABLE IF NOT EXISTS users(
   userid INT PRIMARY KEY,
   fname TEXT,
   lname TEXT,
   superhero TEXT,
   gender TEXT);
"""
cur.execute(sql)#table creation is a write activity in the database. this results iin uncommited change
conn.commit() # this wil cause the database to save any changes that have been done.

The IF NOT EXISTS will help us when reconnecting to the database. The query will allow us to check if the table exists, and if it does, nothing is changed.

# Insert into Table

Let’s take a look at how to add data with SQLite in Python to the database we just created. Similar to the table generation query, the query to add data uses the cursor object to execute the query.

In [197]:
##INSERT INTO users(userid, fname, lname, superhero, gender)- shows the order in which the data should be inserted
# VALUES('00001', 'Bruce', 'Wayne', 'Batman', 'male') - shows the data to be entered in the order specified.
#note the multi line string """ hich allows you to enter a string on multiple lines
sql = """
INSERT INTO users(userid, fname, lname, superhero, gender)
   VALUES('00001', 'Bruce', 'Wayne', 'Batman', 'male');
"""
cur.execute(sql)
conn.commit()

Often, when we’re working within Python, we’ll have variables that hold values for us. For example, we may have a tuple that contains that information about a user which might look like this:

In [198]:
#You can declare a variable that stores the string to put in the data base
# this is useful if you programme is going to be interactive and data is not known in advance and users will have to enter data to populate the table
user = ('00002', 'Sue', 'Storm', 'Invisible Woman', 'female')

If we wanted to load this data into our database, we would use a different convention:



In [199]:
#??? is a place holder foe the fields in the database
cur.execute("INSERT INTO users VALUES(?, ?, ?, ?, ?);", user)
conn.commit()

Incidentally, using the (?, ?, …) method we noted above also helps protect against SQL injection attacks.

It’s important to note here that the SQLite expects the values to be in tuple-format. However, the variable can contain a list, as long as the list items are tuples. For example, we could add more users using the variable:

In [200]:
#in this case the database will read multiple tuples in the list. this will be alist of entries or rows
more_users = [('00003', 'Peter', 'Parker', 'Spider Man', 'male'), ('00004', 'Tony', 'Stark', 'Ironman', 'male'), ('00005', 'Diana', 'Prince', 'Wonder Woman', 'female')]

In [201]:
cur.executemany("INSERT INTO users VALUES(?, ?, ?, ?, ?);", more_users)
conn.commit()

In this case, instead of using the execute function, we’ll want to use the executemany function

## Fetching Data

Here we take a look at how to select data with SQLite in Python! We’ll follow a similar structure as we did to execute queries above

In [202]:
sql = '''
SELECT * FROM users;
'''
rows = cur.execute(sql)
for row in rows:
  print(row)

(1, 'Bruce', 'Wayne', 'Batman', 'male')
(2, 'Sue', 'Storm', 'Invisible Woman', 'female')
(3, 'Peter', 'Parker', 'Spider Man', 'male')
(4, 'Tony', 'Stark', 'Ironman', 'male')
(5, 'Diana', 'Prince', 'Wonder Woman', 'female')


In [203]:
#fetchone is a function property of the cursor
sql = '''
SELECT * FROM users;
'''
cur.execute(sql)
one_result = cur.fetchone()
print(one_result)

(1, 'Bruce', 'Wayne', 'Batman', 'male')


We wanted to return more than only one result, we could use the fetchmany()

In [204]:
sql = '''
SELECT * FROM users;
'''
cur.execute(sql)
three_results = cur.fetchmany(3)
print(three_results)

[(1, 'Bruce', 'Wayne', 'Batman', 'male'), (2, 'Sue', 'Storm', 'Invisible Woman', 'female'), (3, 'Peter', 'Parker', 'Spider Man', 'male')]


We canuse the fetchall() function to return all the results.

In [205]:
sql = '''
SELECT * FROM users;
'''
cur.execute(sql)
all_results = cur.fetchall()
print(all_results)

[(1, 'Bruce', 'Wayne', 'Batman', 'male'), (2, 'Sue', 'Storm', 'Invisible Woman', 'female'), (3, 'Peter', 'Parker', 'Spider Man', 'male'), (4, 'Tony', 'Stark', 'Ironman', 'male'), (5, 'Diana', 'Prince', 'Wonder Woman', 'female')]


In [206]:
sql = '''
SELECT * FROM users;
'''
cur.execute(sql)
all_results = cur.fetchall()
print(all_results)

[(1, 'Bruce', 'Wayne', 'Batman', 'male'), (2, 'Sue', 'Storm', 'Invisible Woman', 'female'), (3, 'Peter', 'Parker', 'Spider Man', 'male'), (4, 'Tony', 'Stark', 'Ironman', 'male'), (5, 'Diana', 'Prince', 'Wonder Woman', 'female')]


## Delete Data

In [207]:
sql='''
DELETE FROM users WHERE lname='Parker';
'''
cur.execute(sql)
conn.commit()

Did it work?

In [220]:
#print to see if above delete command was successful
sql='''
select * from users where lname='Parker';
'''
sql1='''SELECT * FROM users'''
cur.execute(sql1)
all_results=cur.fetchall()
for row in all_results:
   print(row)

(1, 'Bruce', 'Wayne', 'Batman', 'male')
(2, 'Sue', 'Storm', 'Invisible Woman', 'female')
(4, 'Tony', 'Stark', 'Ironman', 'male')
(5, 'Diana', 'Prince', 'Wonder Woman', 'female')


In [221]:
#This prints out an empty list, confirming that the record has been deleted.
sql='''
select * from users where lname='Parker';
'''
cur.execute(sql)
cur.fetchall()

[]

This prints out an empty list, confirming that the record has been deleted.

## pandas

We can use the pandas package to read a SQLite database

In [222]:
import pandas as pd

sql='''
SELECT * from users;
'''
# Load the data into a DataFrame
users_df = pd.read_sql_query(sql, conn)
users_df

,userid,fname,lname,superhero,gender
0,1,Bruce,Wayne,Batman,male
1,2,Sue,Storm,Invisible Woman,female
2,4,Tony,Stark,Ironman,male
3,5,Diana,Prince,Wonder Woman,female


and create table from a dataframe

In [210]:
# Select only data for females
users_feamle_df = users_df[users_df.gender == 'female']

# Write the new DataFrame to a new SQLite table
users_feamle_df.to_sql("females", conn, if_exists="replace")

2

## View Schema

Was the new table added.  Lets have a look at the schema.

In [225]:
#this shows the list of tables in the database. We have the users and females tables
#this section refers to a database sqllite master
sql='''
SELECT name FROM sqlite_master WHERE type='table';
'''
cur.execute(sql)
cur.fetchall()

[('users',), ('females',)]

Now lets get details on a table

In [224]:
sql='''
PRAGMA table_info('users');
'''
cur.execute(sql)
cur.fetchall()

[(0, 'userid', 'INT', 0, None, 1),
 (1, 'fname', 'TEXT', 0, None, 0),
 (2, 'lname', 'TEXT', 0, None, 0),
 (3, 'superhero', 'TEXT', 0, None, 0),
 (4, 'gender', 'TEXT', 0, None, 0)]

## Delete (Drop) Table

When the IF EXISTS clause is used along with DROP TABLE syntax SQLite will not report any error message if the table does not exist.

In [213]:
#Dropig the table removes it from the databse.

#sql='''
#DROP TABLE IF EXISTS users;
#'''
#cur.execute(sql)
#cur.fetchall()


Did it work?

In [214]:
#We should have only the fenales table avaialable
sql='''
SELECT name FROM sqlite_master WHERE type='table';
'''
cur.execute(sql)
cur.fetchall()

[('users',), ('females',)]

## Close the connection

The connection must be closed at the end of the session

In [215]:
#conn.close()